In [ ]:
import pandas as pd
import numpy as np

# Preprocesamiento

In [ ]:
venancio = pd.read_parquet('../expresion/venancio.parquet')
venancio

In [ ]:
condicion = pd.read_csv('../estres/condicion.csv', index_col=0)
condicion

In [ ]:
joined = venancio.join(condicion)
joined = joined[joined['condicion'].notna()]
joined

In [ ]:
X = joined.drop('condicion', axis=1)
y = joined.condicion

# Modelado

In [ ]:
from sklearn import model_selection, ensemble, manifold
import matplotlib.pyplot as plt

In [ ]:
y = y == 'control'
y

In [ ]:
tsne = manifold.TSNE()
X_tsne = tsne.fit_transform(X)

In [ ]:
plt.scatter(X_tsne[y, 0], X_tsne[y, 1], alpha=0.5, label='control')
plt.scatter(X_tsne[~y, 0], X_tsne[~y, 1], alpha=0.5, label='estrés')
plt.legend()
plt.title('t-SNE expresión genes')

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y)

In [ ]:
random_forest = ensemble.RandomForestClassifier()
params = {
    'n_estimators': np.arange(100, 1000, 100),
    'max_depth': np.arange(20, 50, 10),
    'min_samples_split': np.arange(2, 15, 4),
    'min_samples_leaf': np.arange(2, 15, 4),
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True]
}

In [ ]:
search = model_selection.RandomizedSearchCV(estimator=random_forest, param_distributions=params, n_jobs=-1)
search.fit(X_train, y_train)

In [ ]:
search.cv_results_['mean_test_score'].mean()

In [ ]:
estimator = search.best_estimator_
importances = np.flip(np.sort(estimator.feature_importances_))
importances

In [ ]:
plt.plot(importances)
plt.title('Importancias de features ordenadas (Random Forest)')

In [ ]:
X.columns[estimator.feature_importances_ > 0]

In [ ]:
X_tsne = tsne.fit_transform(X.loc[:, estimator.feature_importances_ > 0.001])

In [ ]:
model_selection.LearningCurveDisplay.from_estimator(estimator, X_train, y_train)

In [ ]:
plt.scatter(X_tsne[y, 0], X_tsne[y, 1], alpha=0.5, label='control')
plt.scatter(X_tsne[~y, 0], X_tsne[~y, 1], alpha=0.5, label='estrés')
plt.legend()
plt.title('t-SNE expresión genes seleccionados')

In [ ]:
gbr = ensemble.GradientBoostingClassifier()
params = {
    "loss": ['log_loss', 'exponential'],
    "learning_rate": np.arange(0, 10, 1),
    "n_estimators": np.arange(1, 200, 1),
    "max_depth": np.arange(1, 50, 1),
    "max_features": ['sqrt', 'log2']
}

In [ ]:
search = model_selection.RandomizedSearchCV(gbr, param_distributions=params, n_jobs=-1)
search.fit(X_train, y_train)

In [ ]:
search.cv_results_['mean_test_score'].mean()

In [ ]:
model_selection.LearningCurveDisplay.from_estimator(gbr, X_train, y_train)